# Ch05. Deep Learning Computation
## 5.1 Layers and Blocks

In [1]:
from mxnet import np,npx
from mxnet.gluon import nn
npx.set_np()

In [9]:
net=nn.Sequential()
net.add(nn.Dense(256,activation='relu'))
net.add(nn.Dense(10))
net.initialize()

X=np.random.uniform(size=(2,20))
print(X)
net(X)

[[0.5244929  0.27717388 0.45943195 0.26419154 0.62452465 0.07948767
  0.729858   0.82709134 0.9121407  0.6325586  0.58385587 0.02031024
  0.9023594  0.05920758 0.9929139  0.33857688 0.54573417 0.3061341
  0.25212002 0.87476176]
 [0.6425134  0.56946045 0.7291909  0.09361915 0.64827555 0.09434061
  0.05570889 0.64150625 0.18864442 0.37547633 0.23858361 0.9407423
  0.2450219  0.15967701 0.9391073  0.8505674  0.6680149  0.18208402
  0.2321935  0.31100443]]


array([[ 0.11196154, -0.03203352, -0.11136746, -0.00042839,  0.00577225,
         0.02923119, -0.03771511, -0.05057289,  0.012499  , -0.07507581],
       [ 0.09532058, -0.04891215, -0.08393399, -0.03934758, -0.01236376,
         0.07096371, -0.05273829, -0.03675307,  0.07640459, -0.04559512]])

In [4]:
# 5.1.1 A Custom Block
class MLP(nn.Block):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.hidden=nn.Dense(256,activation='relu')
        self.out=nn.Dense(10)
    def forward(self,X):
        return self.out(self.hidden(X))

In [5]:
net=MLP()
net.initialize()
net(X)

array([[ 0.03333075, -0.00020987,  0.0087921 , -0.02916729,  0.0503433 ,
         0.04706917,  0.04869975, -0.01487542, -0.00142947,  0.01638612],
       [ 0.0372573 , -0.01963825, -0.04683002, -0.03069271,  0.01393091,
         0.04475477,  0.02624594, -0.00272092, -0.02361028,  0.01886239]])

In [6]:
# 5.1.2 The Sequential Block
class MySequential(nn.Block):
    def add(self, block):
        self._children[block.name]=block
    def forward(self,X):
        for block in self._children.values():
            X=block(X)
        return X

In [11]:
net=MySequential()
net.add(nn.Dense(256,activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(X)

array([[ 0.01456538, -0.0146833 ,  0.02022494, -0.05011837,  0.04310356,
         0.03839159, -0.00798961, -0.00444266, -0.05437186,  0.02408027],
       [ 0.04924705,  0.006925  ,  0.05610218, -0.05266779,  0.01618375,
         0.06415477,  0.00700657, -0.03358176, -0.0182515 ,  0.00162864]])

In [12]:
# 5.1.3 Executing Code in the Forward Propagation Function
class FixedHiddenMLP(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.rand_weight = self.params.get_constant('rand_weight', 
                                                    np.random.uniform(size=(20, 20)))
        self.dense = nn.Dense(20, activation='relu')

    def forward(self, X):
        X = self.dense(X)
        X = npx.relu(np.dot(X, self.rand_weight.data()) + 1)
        X = self.dense(X)
        while np.abs(X).sum() > 1:
            X /= 2
        return X.sum()

In [13]:
net = FixedHiddenMLP()
net.initialize()
net(X)

array(0.510016)

In [14]:
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation='relu'),
                     nn.Dense(32, activation='relu'))
        self.dense = nn.Dense(16, activation='relu')

    def forward(self, X):
        return self.dense(self.net(X))

In [33]:
chimera = nn.Sequential()
chimera.add(NestMLP(), nn.Dense(20), FixedHiddenMLP())
chimera.initialize()
chimera(X)

array(0.9472705)

In [ ]:
# 5.1.4 Efficiency